In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]

In [3]:
from langchain_mistralai import ChatMistralAI

chatModel = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

## Prompt Template

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}."
)

llmModelPrompt = prompt_template.format(
    adjective="curious", 
    topic="the Kennedy family"
)


In [5]:

response = chatModel.invoke(llmModelPrompt)

In [6]:
response.content

'Sure, I\'d be happy to share a curious story about the Kennedy family. This one involves a secret letter from Jackie Kennedy to the Soviet leader Nikita Khrushchev.\n\nAfter the assassination of President John F. Kennedy in 1963, his widow, Jacqueline Kennedy, was understandably grief-stricken. What\'s less known is that she took it upon herself to write a personal letter to the Soviet leader, Nikita Khrushchev. This was at the height of the Cold War, and Khrushchev was often portrayed as an enemy of the United States.\n\nIn her letter, Jackie thanked Khrushchev for a condolence letter he had sent after JFK\'s death. She also expressed her and her late husband\'s desire for peace, writing, "He was bending all his energies toward this one goal. Then suddenly, the curtain which had fallen between the two of us lifted. We saw each other, we exchanged messages and gifts, and we had become friends."\n\nThe letter was considered so sensitive that it was kept secret for decades. It was final

## Chat Prompt Template

In [7]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Historian",
    topic="The Kennedy family",
    user_input="How many grandchildren had Joseph P. Kennedy?"
)



In [8]:
response = chatModel.invoke(messages)

In [9]:
response.content

"Joseph P. Kennedy Sr. had nine children with his wife Rose Fitzgerald Kennedy. These children were, in order of birth: Joseph P. Kennedy Jr., John F. Kennedy, Rosemary Kennedy, Kathleen Cavendish (née Kennedy), Eunice Kennedy Shriver, Patricia Kennedy Lawford, Robert F. Kennedy, Jean Kennedy Smith, and Edward M. Kennedy.\n\nThe number of grandchildren Joseph P. Kennedy Sr. had can be calculated by adding up the children of each of his offspring. Here's a breakdown:\n\n1. Joseph P. Kennedy Jr. did not have any children.\n2. John F. Kennedy had three children: Caroline Kennedy, John F. Kennedy Jr., and Patrick Bouvier Kennedy (who died in infancy).\n3. Rosemary Kennedy did not have any children.\n4. Kathleen Cavendish had one child: Victoria Francis Pakenham.\n5. Eunice Kennedy Shriver had five children: Robert Sargent Shriver III, Maria Shriver, Timothy Shriver, Mark Shriver, and Anthony Shriver.\n6. Patricia Kennedy Lawford had four children: Christopher Kennedy Lawford, Sydney Maleia

## Fewshort template

In [10]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [11]:
chain = final_prompt | chatModel

response = chain.invoke({"input": "Good Night"})

In [12]:
response.content

'Buenas noches'

## With Parser

In [14]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | chatModel | json_parser



In [15]:
response = json_chain.invoke({"question": "What is the biggest country?"})

In [16]:
response

{'answer': 'Russia is the biggest country by land area.'}